## Smoking Cessation Study

In [1]:
import numpy as np                                      #standard imports
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import savReaderWriter as srw                           #Python package for reading SPSS .sav files
import seaborn as sb
import datetime as dt

## Read the manually generated enrollment data with censoring flags

In [2]:
subs = pd.read_csv('../enrolled_with_flags.csv',parse_dates=['term','withdrew','susp','intake','fcr'])
print(subs.shape)
subs.dtypes

(406, 12)


isg_no                 float64
intake          datetime64[ns]
fcr             datetime64[ns]
hours_to_fcr           float64
term            datetime64[ns]
withdrew        datetime64[ns]
susp            datetime64[ns]
days                   float64
cp_3                   float64
react                  float64
flag                   float64
event                  float64
dtype: object

### Compute the time for the end of suspension, if the subject was reinstated

In [3]:
subs['end_susp'] = subs['susp'] + pd.TimedeltaIndex(subs['days'], unit='D')
subs.dtypes

isg_no                 float64
intake          datetime64[ns]
fcr             datetime64[ns]
hours_to_fcr           float64
term            datetime64[ns]
withdrew        datetime64[ns]
susp            datetime64[ns]
days                   float64
cp_3                   float64
react                  float64
flag                   float64
event                  float64
end_susp        datetime64[ns]
dtype: object

### Compute the time subject would be censored out after 87 days if they had not reported a first cigarette

In [4]:
subs['day87'] = subs['intake'] + + dt.timedelta(days=87)

subs.dtypes

isg_no                 float64
intake          datetime64[ns]
fcr             datetime64[ns]
hours_to_fcr           float64
term            datetime64[ns]
withdrew        datetime64[ns]
susp            datetime64[ns]
days                   float64
cp_3                   float64
react                  float64
flag                   float64
event                  float64
end_susp        datetime64[ns]
day87           datetime64[ns]
dtype: object

### Compute the time to event or censoring as the earliest of the following times:

1.  The time the first cigarette was reported - event at this time
2.  The time the subject withdrew - censored out at this time if first cigarette not reported
3.  The time the subject was terminated - censored out at this time if first cigarette not reported
4.  87 days after intake - subject censored out without reporting a first cigarette

Note: Per Beau, subjects who are reinstated, terminated, and report a first cigarette on the same day can be included.  These were treated as if the suspension didn't happen and the subject reported a first cigarette and was terminated on the same day.

In [5]:
subs['event_time'] = subs[['fcr','day87','term','withdrew']].min(axis=1)

### Compute the difference between intake and event or censoring in hours

In [6]:
subs['hours_to_event'] =((subs.event_time-subs.intake).astype('timedelta64[s]')/3600.0)

In [7]:
subs

,isg_no,intake,fcr,hours_to_fcr,term,withdrew,susp,days,cp_3,react,flag,event,end_susp,day87,event_time,hours_to_event
0,10587.0,2010-09-27 12:11:00,2010-09-28 13:48:00,25.616667,NaT,NaT,NaT,NaN,1.0,NaN,0.0,1.0,NaT,2010-12-23 12:11:00,2010-09-28 13:48:00,25.616667
1,10636.0,2010-10-07 11:38:36,2010-10-08 13:34:59,25.939722,NaT,NaT,NaT,NaN,1.0,NaN,0.0,1.0,NaT,2011-01-02 11:38:36,2010-10-08 13:34:59,25.939722
2,10657.0,2010-10-21 10:04:59,2010-10-21 13:04:00,2.983611,NaT,NaT,NaT,NaN,1.0,NaN,0.0,1.0,NaT,2011-01-16 10:04:59,2010-10-21 13:04:00,2.983611
3,10664.0,2010-10-22 13:27:39,NaT,NaN,NaT,NaT,NaT,NaN,1.0,NaN,1.0,0.0,NaT,2011-01-17 13:27:39,2011-01-17 13:27:39,2088.000000
4,10671.0,2010-10-22 14:34:21,2010-10-27 09:19:00,114.744167,NaT,NaT,NaT,NaN,1.0,NaN,0.0,1.0,NaT,2011-01-17 14:34:21,2010-10-27 09:19:00,114.744167
5,10678.0,2010-10-25 11:51:46,2010-11-25 21:10:00,754.303889,NaT,NaT,NaT,NaN,1.0,NaN,0.0,1.0,NaT,2011-01-20 11:51:46,2010-11-25 21:10:00,753.303889
6,10685.0,2010-10-25 13:24:33,NaT,NaN,NaT,NaT,NaT,NaN,1.0,NaN,1.0,0.0,NaT,2011-01-20 13:24:33,2011-01-20 13:24:33,2088.000000
7,10706.0,2010-10-27 17:42:20,2010-10-27 21:14:00,3.527778,NaT,NaT,NaT,NaN,1.0,NaN,0.0,1.0,NaT,2011-01-22 17:42:20,2010-10-27 21:14:00,3.527778
8,10755.0,2010-11-08 10:03:40,2010-11-08 18:57:00,8.888889,NaT,NaT,NaT,NaN,1.0,NaN,0.0,1.0,NaT,2011-02-03 10:03:40,2010-11-08 18:57:00,8.888889
9,10769.0,2010-11-12 12:32:08,2010-12-07 20:49:00,608.281111,NaT,NaT,NaT,NaN,1.0,NaN,0.0,1.0,NaT,2011-02-07 12:32:08,2010-12-07 20:49:00,608.281111


### Exclude subjects manually marked for exclusion because of data issues or dropping out on the first day

In [8]:
subs['no_flag'] = subs.flag.isnull()

In [9]:
subs_included = subs.loc[subs['no_flag'] == False]
print(subs_included.shape)

(394, 17)


### Create a dataframe with only the columns needed as input for the Cox regression:

1. Identifying number (isg_number)
2. Time to event or censoring in hours
3. Flag indicating event or censor

In [10]:
cox_input = subs_included[['isg_no','event','hours_to_event']]
cox_input

,isg_no,event,hours_to_event
0,10587.0,1.0,25.616667
1,10636.0,1.0,25.939722
2,10657.0,1.0,2.983611
3,10664.0,0.0,2088.000000
4,10671.0,1.0,114.744167
5,10678.0,1.0,753.303889
6,10685.0,0.0,2088.000000
7,10706.0,1.0,3.527778
8,10755.0,1.0,8.888889
9,10769.0,1.0,608.281111


## Read the long merged dataset

In [11]:
savFileName = "../SEMIII_isg_long_merged_de_id'd_1.sav"      #March 21 2017 download

## Read data dictionary from .sav file and list keys

In [12]:
with srw.SavHeaderReader(savFileName) as header:       #Read the .sav file metadata
    dd = header.dataDictionary()                       #get a list of the metadata elements
dd.keys()

['valueLabels',
 'varTypes',
 'varSets',
 'varAttributes',
 'varRoles',
 'measureLevels',
 'caseWeightVar',
 'varNames',
 'varLabels',
 'formats',
 'multRespDefs',
 'columnWidths',
 'fileAttributes',
 'alignments',
 'fileLabel',
 'missingValues']

### List the SPSS variable names

In [13]:
dd['varNames']

['isg_no',
 'CalledNumber',
 'CreationDate',
 'OriginalCalloutTime',
 'CallDate',
 'chartreview_a1_012',
 'discharge_equals_1stcall',
 'chartreview_a1_012a',
 'Note',
 'intakesurvey',
 'intakesurveyoutoforder',
 'intakesurveymissing',
 'eodsurvey',
 'firstcigarettereported',
 'firstcigarettesurvey',
 'extracall',
 'isgmalfunction',
 'suspended',
 'CallDayNumber',
 'CallType',
 'CalloutTime',
 'CallResultTypeID',
 'CallResult',
 'recodedcallresult',
 'Attempts',
 'V24',
 'V25',
 'V26',
 'V27',
 'V28',
 'V29',
 'V30',
 'V31',
 'V32',
 'V33',
 'V34',
 'V35',
 'V36',
 'V37',
 'V38',
 'V39',
 'V40',
 'V41',
 'V42',
 'V43',
 'V44',
 'V45',
 'V46',
 'V47',
 'V48',
 'V49',
 'V50',
 'V51',
 'V52',
 'V53',
 'V54',
 'V55',
 'V56',
 'V57',
 'V58',
 'V59',
 'V60',
 'V61',
 'V62',
 'V63',
 'V64',
 'V65',
 'V66',
 'V67',
 'V68',
 'V69',
 'V70',
 'V71',
 'V72',
 'V73',
 'V74',
 'V75',
 'V76',
 'V77',
 'V78',
 'V79',
 'V80',
 'V81',
 'V82',
 'V83',
 'V84',
 'V85',
 'V86',
 'V87',
 'V88',
 'V89',
 'V90'

### List the SPSS variable labels

In [14]:
dd['varLabels']

{'Attempts': '',
 'CR_Location_recode': 'CR Location Recoded',
 'CallDate': '',
 'CallDayNumber': 'Call Day Number',
 'CallResult': '',
 'CallResultTypeID': '',
 'CallType': 'Call Type',
 'CalledNumber': '',
 'CalloutTime': '',
 'CreationDate': '',
 'Note': '',
 'OriginalCalloutTime': '',
 'V100': '79.some other activity?',
 'V101': '80.at home?',
 'V102': '81.at work?',
 'V103': "82.at someone else's home?",
 'V104': '83.a bar or restaurant?',
 'V105': '84.in a car?',
 'V106': '85.outside?',
 'V107': '86.at the hospital?',
 'V108': '87.some other location?',
 'V109': '89.Do you think you smoked because you were coping with stress or some other negative emotion?',
 'V110': '90.Today, I had problems with my health.',
 'V111': '91.Today, I felt like I had a serious illness.',
 'V112': '92.Today, I was afraid when I thought about my health.',
 'V113': '93.My health made me sad today.',
 'V114': '94.Today, my health made me anxious or nervous.',
 'V115': '95.Rate the degree to which you fe

## Make a list of the column names and initialize a dictionary to hold the data

In [15]:
columns=[]                                   #list for column names
datad={}
               
for name in dd['varNames']:                  #varNames are bytes
    colname = name.decode()                  #convert to string
    columns.append(colname)                  #add to list
    datad[colname] = []                      #initialize dictionary for data with empty lists

## Read the data and fill in the dictionary

In [16]:
with srw.SavReader(savFileName) as reader:      #use the savReaderWriter package to read the .sav contents

    for line in reader:                         #loop through the cases
        for i in range(len(columns)):           #store each data value in the data dictionary
            value = line[i]
            datad[columns[i]].append(value)

### Create a data frame from the dictionary

In [17]:
mcdf = pd.DataFrame(datad,columns=columns)      #create a pandas data frame from the data dictionary
print(mcdf.shape)
mcdf.head()                                     #show the first 5 rows

(66760, 511)


,isg_no,CalledNumber,CreationDate,OriginalCalloutTime,CallDate,chartreview_a1_012,discharge_equals_1stcall,chartreview_a1_012a,Note,intakesurvey,...,baseline_b1_003b,baseline_c1_003b,baseline_d1_003b,p_sev_1,p_sev_2,p_sev_3,p_intnt,c_intnt,smk_ca,CR_Location_recode
0,10573.0,4.015235e+09,2010-09-18 08:44:03,2010-09-18 08:43:42,2010-09-18,2010-09-18,1.0,13:59,intake,1.0,...,4.0,2.0,4.0,3.75,3.25,2.75,1.5,5.0,4.5,3.0
1,10573.0,4.015235e+09,2010-09-18 19:31:22,2010-09-18 19:31:00,2010-09-18,None,NaN,,,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10573.0,4.015235e+09,2010-09-18 20:05:29,2010-09-18 20:05:00,2010-09-18,None,NaN,,,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10573.0,4.015235e+09,2010-09-18 20:32:06,2010-09-18 20:32:00,2010-09-18,None,NaN,,,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10573.0,4.015235e+09,2010-09-18 20:56:11,2010-09-18 20:56:00,2010-09-18,None,NaN,,,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Extract the perceived severity Leikert scale variables

In [18]:
persver = mcdf.loc[mcdf['baseline_b1_001'] < 10000][['subject_id','isg_no',
                'baseline_b1_001','baseline_c1_001','baseline_d1_001',
                'baseline_b1_002','baseline_c1_002','baseline_d1_002',
                'baseline_b1_003','baseline_c1_003','baseline_d1_003',
                'baseline_b1_004','baseline_c1_004','baseline_d1_004']]
    
print(persver.shape)
persver.head()

(436, 14)


,subject_id,isg_no,baseline_b1_001,baseline_c1_001,baseline_d1_001,baseline_b1_002,baseline_c1_002,baseline_d1_002,baseline_b1_003,baseline_c1_003,baseline_d1_003,baseline_b1_004,baseline_c1_004,baseline_d1_004
0,522.0,10573.0,5.0,5.0,3.0,5.0,5.0,3.0,2.0,4.0,2.0,1.0,1.0,1.0
89,21.0,10580.0,3.0,3.0,3.0,3.0,3.0,3.0,2.0,5.0,3.0,2.0,2.0,3.0
130,22.0,10587.0,5.0,4.0,4.0,1.0,3.0,2.0,3.0,3.0,1.0,4.0,2.0,4.0
357,23.0,10636.0,4.0,4.0,4.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
579,523.0,10643.0,4.0,4.0,2.0,4.0,4.0,2.0,3.0,3.0,4.0,5.0,5.0,4.0


### Replace those pesky 9999.0 values with missing values

In [19]:
persver.replace(9999.0, np.nan,inplace=True) 

### Examine the result

In [20]:
persver.describe()

,subject_id,isg_no,baseline_b1_001,baseline_c1_001,baseline_d1_001,baseline_b1_002,baseline_c1_002,baseline_d1_002,baseline_b1_003,baseline_c1_003,baseline_d1_003,baseline_b1_004,baseline_c1_004,baseline_d1_004
count,436.000000,436.000000,431.000000,429.000000,430.000000,431.000000,429.000000,430.000000,431.000000,429.000000,430.000000,431.000000,429.000000,430.000000
mean,361.360092,12641.424312,3.770302,4.174825,3.346512,3.603248,3.969697,3.218605,2.545244,2.142191,2.306977,2.986079,3.475524,3.237209
std,249.621960,6011.769384,1.362504,1.032065,1.378633,1.352994,1.154639,1.351379,1.356181,1.178394,1.237976,1.481605,1.386680,1.423909
min,21.000000,10573.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,129.750000,11418.250000,3.000000,4.000000,2.000000,3.000000,3.000000,2.000000,1.000000,1.000000,1.000000,2.000000,2.000000,2.000000
50%,239.500000,12256.500000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,2.000000,2.000000,2.000000,3.000000,4.000000,4.000000
75%,613.250000,13080.750000,5.000000,5.000000,4.000000,5.000000,5.000000,4.000000,4.000000,3.000000,3.000000,4.000000,5.000000,4.000000
max,722.000000,99999.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


### Extract the quit intentions Leikert scale variables

In [21]:
quint = mcdf.loc[mcdf['baseline_f1_002'] < 10000][['subject_id','isg_no',
                'baseline_f1_002','baseline_f1_004','baseline_f1_006','baseline_f1_007','baseline_f1_008',
                'baseline_f2_002','baseline_f2_004','baseline_f2_006','baseline_f2_007','baseline_f2_008']]
                
print(quint.shape)
quint.head()

(436, 12)


,subject_id,isg_no,baseline_f1_002,baseline_f1_004,baseline_f1_006,baseline_f1_007,baseline_f1_008,baseline_f2_002,baseline_f2_004,baseline_f2_006,baseline_f2_007,baseline_f2_008
0,522.0,10573.0,2.0,2.0,1.0,1.0,5.0,5.0,5.0,5.0,5.0,1.0
89,21.0,10580.0,3.0,5.0,2.0,4.0,1.0,4.0,4.0,1.0,3.0,1.0
130,22.0,10587.0,2.0,3.0,1.0,1.0,1.0,3.0,3.0,4.0,1.0,5.0
357,23.0,10636.0,4.0,4.0,3.0,4.0,3.0,4.0,4.0,3.0,3.0,3.0
579,523.0,10643.0,4.0,4.0,4.0,4.0,2.0,4.0,4.0,4.0,4.0,2.0


### Replace those pesky 9999.0 values with missing values

In [22]:
quint.replace(9999.0, np.nan,inplace=True) 

### Examine the result

In [23]:
quint.describe()

,subject_id,isg_no,baseline_f1_002,baseline_f1_004,baseline_f1_006,baseline_f1_007,baseline_f1_008,baseline_f2_002,baseline_f2_004,baseline_f2_006,baseline_f2_007,baseline_f2_008
count,436.000000,436.000000,430.000000,430.000000,430.000000,430.000000,430.000000,429.000000,430.000000,430.000000,429.000000,430.000000
mean,361.360092,12641.424312,2.746512,3.141860,2.658140,3.081395,2.806977,3.878788,3.872093,3.379070,3.307692,2.248837
std,249.621960,6011.769384,1.365039,1.445471,1.432837,1.446925,1.369820,1.233960,1.257115,1.410262,1.465638,1.320781
min,21.000000,10573.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,129.750000,11418.250000,1.000000,2.000000,1.000000,2.000000,2.000000,3.000000,3.000000,2.000000,2.000000,1.000000
50%,239.500000,12256.500000,3.000000,3.000000,3.000000,3.000000,3.000000,4.000000,4.000000,4.000000,4.000000,2.000000
75%,613.250000,13080.750000,4.000000,4.000000,4.000000,4.000000,4.000000,5.000000,5.000000,5.000000,5.000000,3.000000
max,722.000000,99999.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


### Extract the causal attribution Leikert scale variables

In [24]:
causatt = mcdf.loc[mcdf['baseline_e1_004'] < 10000][['subject_id','isg_no',
                'baseline_e1_004','baseline_e2_001','baseline_e2_002','baseline_e2_003','baseline_e2_004']]

print(causatt.shape)
causatt.head()

(436, 7)


,subject_id,isg_no,baseline_e1_004,baseline_e2_001,baseline_e2_002,baseline_e2_003,baseline_e2_004
0,522.0,10573.0,5.0,5.0,4.0,5.0,4.0
89,21.0,10580.0,3.0,3.0,3.0,3.0,3.0
130,22.0,10587.0,5.0,2.0,2.0,4.0,3.0
357,23.0,10636.0,3.0,5.0,5.0,5.0,4.0
579,523.0,10643.0,5.0,5.0,5.0,2.0,3.0


### Replace those pesky 9999.0 values with missing values

In [25]:
causatt.replace(9999.0, np.nan,inplace=True) 

### Examine the result

In [26]:
causatt.describe()

,subject_id,isg_no,baseline_e1_004,baseline_e2_001,baseline_e2_002,baseline_e2_003,baseline_e2_004
count,436.000000,436.000000,429.000000,430.000000,430.000000,430.000000,429.000000
mean,361.360092,12641.424312,3.909091,3.532558,3.827907,4.444186,3.580420
std,249.621960,6011.769384,1.252525,1.393495,1.250888,0.984922,1.205978
min,21.000000,10573.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,129.750000,11418.250000,3.000000,3.000000,3.000000,4.000000,3.000000
50%,239.500000,12256.500000,4.000000,4.000000,4.000000,5.000000,4.000000
75%,613.250000,13080.750000,5.000000,5.000000,5.000000,5.000000,5.000000
max,722.000000,99999.000000,5.000000,5.000000,5.000000,5.000000,5.000000


### Extract the event related fear Leikert scale variables

In [27]:
erfear = mcdf.loc[mcdf['baseline_b2_002'] < 10000][['subject_id','isg_no',
                'baseline_b2_002','baseline_c2_002','baseline_d2_002']]

print(erfear.shape)
erfear.head()

(436, 5)


,subject_id,isg_no,baseline_b2_002,baseline_c2_002,baseline_d2_002
0,522.0,10573.0,5.0,1.0,1.0
89,21.0,10580.0,4.0,5.0,3.0
130,22.0,10587.0,4.0,4.0,1.0
357,23.0,10636.0,4.0,3.0,2.0
579,523.0,10643.0,1.0,1.0,1.0


### Replace those pesky 9999.0 values with missing values

In [28]:
erfear.replace(9999.0, np.nan,inplace=True) 

### Examine the result

In [29]:
erfear.describe()

,subject_id,isg_no,baseline_b2_002,baseline_c2_002,baseline_d2_002
count,436.000000,436.000000,431.000000,430.000000,430.000000
mean,361.360092,12641.424312,2.858469,2.930233,2.069767
std,249.621960,6011.769384,1.582144,1.586589,1.440269
min,21.000000,10573.000000,1.000000,1.000000,1.000000
25%,129.750000,11418.250000,1.000000,1.000000,1.000000
50%,239.500000,12256.500000,3.000000,3.000000,1.000000
75%,613.250000,13080.750000,4.000000,5.000000,3.000000
max,722.000000,99999.000000,5.000000,5.000000,5.000000


In [30]:
mrg1 = pd.merge(cox_input,persver,on=['isg_no'],how="left")
mrg1.shape

(394, 16)

In [31]:
mrg2 = pd.merge(mrg1,quint,on=['isg_no'],how="left")
print(mrg2.shape)

(394, 27)


In [32]:
mrg3 = pd.merge(mrg2,causatt,on=['isg_no'],how="left")
print(mrg3.shape)

(394, 33)


In [33]:
mrg4 = pd.merge(mrg3,erfear,on=['isg_no'],how="left")
print(mrg4.shape)
mrg4.head()

(394, 37)


,isg_no,event,hours_to_event,subject_id_x,baseline_b1_001,baseline_c1_001,baseline_d1_001,baseline_b1_002,baseline_c1_002,baseline_d1_002,...,subject_id_x,baseline_e1_004,baseline_e2_001,baseline_e2_002,baseline_e2_003,baseline_e2_004,subject_id_y,baseline_b2_002,baseline_c2_002,baseline_d2_002
0,10587.0,1.0,25.616667,22.0,5.0,4.0,4.0,1.0,3.0,2.0,...,22.0,5.0,2.0,2.0,4.0,3.0,22.0,4.0,4.0,1.0
1,10636.0,1.0,25.939722,23.0,4.0,4.0,4.0,4.0,3.0,3.0,...,23.0,3.0,5.0,5.0,5.0,4.0,23.0,4.0,3.0,2.0
2,10657.0,1.0,2.983611,25.0,4.0,4.0,3.0,3.0,3.0,3.0,...,25.0,4.0,3.0,3.0,3.0,3.0,25.0,4.0,4.0,5.0
3,10664.0,0.0,2088.000000,26.0,2.0,4.0,4.0,2.0,4.0,4.0,...,26.0,4.0,3.0,2.0,3.0,3.0,26.0,2.0,2.0,1.0
4,10671.0,1.0,114.744167,27.0,3.0,3.0,1.0,3.0,3.0,2.0,...,27.0,3.0,3.0,3.0,5.0,3.0,27.0,4.0,4.0,3.0


In [34]:
mrg4.to_csv("../whosein2.csv")